In [ ]:
# ============================================================================
# AGORA CHAT WITH OPENTELEMETRY - MINIMAL VERSION
# ============================================================================

!pip install --force-reinstall git+https://github.com/JerzyKultura/Agora.git
!pip install openai opentelemetry-instrumentation-openai

import os
from openai import OpenAI
from agora.telemetry import AuditLogger, AuditedNode, AuditedFlow

# Auto-instrument OpenAI
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

trace.set_tracer_provider(TracerProvider())
trace.get_tracer_provider().add_span_processor(SimpleSpanProcessor(ConsoleSpanExporter()))
OpenAIInstrumentor().instrument()

# ============================================================================
# LLM + CHAT NODES
# ============================================================================

def call_llm(messages):
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "paste-your-openai-key-here"))
    response = client.chat.completions.create(
        model="gpt-4o-mini", messages=messages, temperature=0.7, max_tokens=500
    )
    return response.choices[0].message.content

class ChatInput(AuditedNode):
    def prep(self, shared):
        if "messages" not in shared:
            shared["messages"] = []
            print("Chat ready. Type 'exit' to quit.")
        return shared["messages"]

    def exec(self, messages):
        user_input = input("\nYou: ").strip()
        return user_input if user_input.lower() != 'exit' else None

    def post(self, shared, prep_res, exec_res):
        if exec_res is None:
            return "exit"
        shared["messages"].append({"role": "user", "content": exec_res})
        return "respond"

class ChatResponse(AuditedNode):
    def prep(self, shared):
        return shared["messages"]

    def exec(self, messages):
        response = call_llm(messages)  # Auto-instrumented
        return response

    def post(self, shared, prep_res, exec_res):
        print(f"\nBot: {exec_res}")
        shared["messages"].append({"role": "assistant", "content": exec_res})
        return "input"

class ChatExit(AuditedNode):
    def exec(self, prep_res):
        print("Goodbye!")
        return None

# ============================================================================
# RUN CHAT
# ============================================================================

def run_chat():
    logger = AuditLogger("chat")

    # Create nodes
    chat_input = ChatInput("Input", logger)
    chat_response = ChatResponse("Response", logger)
    chat_exit = ChatExit("Exit", logger)

    # Build flow
    flow = AuditedFlow("Chat", logger)
    flow.start(chat_input)
    chat_input - "respond" >> chat_response
    chat_input - "exit" >> chat_exit
    chat_response - "input" >> chat_input

    # Run
    shared = {}
    flow.run(shared)

    # Show results
    print(f"\nAudit: {logger.get_summary()}")
    logger.save_json("chat_audit.json")

# Set your API key and run
# os.environ["OPENAI_API_KEY"] = "your-key-here"
# run_chat()

print("Set API key: os.environ['OPENAI_API_KEY'] = 'your-key'")
print("Then run: run_chat()")

  Cloning https://github.com/JerzyKultura/Agora.git to /tmp/pip-req-build-btfr1og6
  Running command git clone --filter=blob:none --quiet https://github.com/JerzyKultura/Agora.git /tmp/pip-req-build-btfr1og6
  Resolved https://github.com/JerzyKultura/Agora.git to commit fa6e758ae2a7df0dd2a34a18f864222915168d67
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for agora: filename=agora-0.1.0-py3-none-any.whl size=7857 sha256=47ede6ce0ca2db8cc4cd9c24eb5cbf1e0be8ff0d2245cc0413b50236642b69bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-vlm47su0/wheels/54/03/09/657d5c3da4e08b83a5da9904f69aa6f4e7fbc9f7660b3580ab
Successfully built agora
  Attempting uninstall: agora
    Found existing installation: agora 0.1.0
    Uninstalling agora-0.1.0:
      Successfully uninstalled agora-0.1.0
Set API key: os.environ['OPENAI_API_KEY'] = 'your-key'
Then run: run_chat()


In [ ]:
run_chat()


Chat ready. Type 'exit' to quit.

You: hi
{
    "name": "node.Input",
    "context": {
        "trace_id": "0x41176f4a34838f98930188b5c63d5efe",
        "span_id": "0x0d55911d515a95d2",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x3d74f2110d144350",
    "start_time": "2025-09-23T12:20:58.644087Z",
    "end_time": "2025-09-23T12:21:03.394237Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "node_name": "Input",
        "node_type": "ChatInput",
        "latency_ms": 4750.128746032715,
        "result_type": "str",
        "batch_size": 2
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "telemetry.sdk.language": "python",
            "telemetry.sdk.name": "opentelemetry",
            "telemetry.sdk.version": "1.37.0",
            "service.name": "unknown_service"
        },
        "schema_url": ""
    }
}
{
    "name": "openai.chat",
    "context": {
        "tra